## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Deploy a Deployment

- The Deployment definition is in the YAML file `manifests/deploy-app.yaml`.

In [1]:
!kubectl apply -f manifests/deploy-app.yaml

deployment.apps/deploy-nginx created


## Let's look at the Deployment's YAML

**Note:**

- The number of `replicas` is set to 3.
- The `revisionHistoryLimit` is set to 3.
- The Pod template's container is listening on `containerPort` 80.
- The Pod template's `labels` are `app: nginx` and `env: prod`
  - The Deployment's `matchLabels` match these labels
    - Therefore, the Deployment will create 3 replicas from the Pod template.
  - The Service's `selector` matches these labels
    - Therefore, the Service will load balance between the three Pod replicas.

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-nginx         # the deployment is named deploy-nginx
spec:
  replicas: 3                 # the deployment defines three replicas
  revisionHistoryLimit: 3     # the deployment defines a maximum of three ReplicaSet revisions to store in history
  selector:
    matchLabels:
      app: nginx              # the deployment has two matchLabels that match the Pod template's labels below
      env: prod               # app = nginx and env = prod
  template:
    metadata:
      name: myapp-pod
      labels:
        app: nginx             # the Pod template has two labels defined (the Service's selector matches these two labels):
        env: prod              # app = nginx and env = prod
    spec:
      containers:
      - name: nginx
        image: nginx:alpine    # the container is based on the nginx:alpine image
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi    
        ports:
        - containerPort: 80    # the Pod template's container is listening on containerPort 80
```

In [2]:
#!type manifests/deploy-app.yaml #use this on Windows
!cat manifests/deploy-app.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-nginx
  labels:
    app: nginx
    env: prod
spec:
  replicas: 3
  revisionHistoryLimit: 3
  selector:
    matchLabels:
      app: nginx
      env: prod
  template:
    metadata:
      labels:
        app: nginx
        env: prod
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi
        ports:
        - containerPort: 80

## Deploy a Service

- The Service definition is in the YAML file `manifests/nodeport.yaml`.

In [3]:
!kubectl apply -f manifests/nodeport.yaml

service/svc-example created


## Let's look at the Service's YAML

**Note:**

- The service's name is `svc-example`.
- The service's is:
  - Listening for external (public) traffic on `nodePort` 32410.
  - Listening for internal (private) traffic on `port` 80.
  - Redirecting traffic to `targetPort` 80.
- The servie's `selector` defines two labels (that match the Pod template's labels for the Deployment):
  - `app: nginx` and `env: prod`
  - This means the Service will (round robin) load balance between the three Pods in the Deployment's ReplicaSet.

```bash
apiVersion: v1
kind: Service
metadata:
  name: svc-example    # the Service's name is svc-example
spec:
  type: NodePort       # the Service's type is NodePort
  selector:
    app: nginx         # the Service's selector defines two labels (the Service's selector matches the Pod template's two labels):
    env: prod          # app = nginx and env = prod
  ports:
  - nodePort: 32410    # the Service is listening for external (public) traffic on nodePort 32410
    protocol: TCP      # the Service's protocol is TCP
    port: 80           # the Service is listening for internal (private) traffic on port 80
    targetPort: 80     # the Service is redirecting traffic to targetPort 80
```

In [4]:
#!type manifests/nodeport.yaml #use this on Windows
!cat manifests/nodeport.yaml

apiVersion: v1
kind: Service
metadata:
  name: svc-example
spec:
  type: NodePort
  selector:
    app: nginx
    env: prod
  ports:
  - nodePort: 32410
    protocol: TCP
    port: 80
    targetPort: 80

## List Pods

- We see that three Pods are running.

In [5]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                            READY   STATUS    RESTARTS   AGE   IP           NODE           NOMINATED NODE   READINESS GATES
deploy-nginx-7b944585d7-78n7r   1/1     Running   0          16s   10.244.1.8   minikube-m02   <none>           <none>
deploy-nginx-7b944585d7-fpqp7   1/1     Running   0          16s   10.244.2.7   minikube-m03   <none>           <none>
deploy-nginx-7b944585d7-x9tl5   1/1     Running   0          16s   10.244.0.9   minikube       <none>           <none>


## List Services

- Notice the `TYPE` is `NodePort` for the `svc-example` Service (with `nodePort` = `32410`).
  - This means any of the nodes' IP addreses can be used to access the cluster externally (publicly) using port `32410`.

In [6]:
#!kubectl get svc -o wide
!kubectl get services -o wide

NAME          TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)        AGE   SELECTOR
kubernetes    ClusterIP   10.96.0.1        <none>        443/TCP        11h   <none>
svc-example   NodePort    10.106.244.119   <none>        80:32410/TCP   16s   app=nginx,env=prod


## List Nodes

- Since we have a `NodePort` Service:
  - Any of the nodes' IP addresses (`INTERNAL-IP`) provide public access to the cluster using `nodePort` `32410`.

In [7]:
!kubectl get nodes -o wide

NAME           STATUS   ROLES           AGE   VERSION   INTERNAL-IP    EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION                       CONTAINER-RUNTIME
minikube       Ready    control-plane   11h   v1.28.3   192.168.49.2   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7
minikube-m02   Ready    <none>          12m   v1.28.3   192.168.49.3   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7
minikube-m03   Ready    <none>          12m   v1.28.3   192.168.49.4   <none>        Ubuntu 22.04.3 LTS   5.15.133.1-microsoft-standard-WSL2   docker://24.0.7


## Access Service

- Use any node's IP address (`INTERNAL-IP`) and the NodePort number: `http://IP:NodePort`
  - If the NodePort number was not explicitly defined, you can get the assigned NodePort number with the command below:
    - `kubectl get service svc-example --output='jsonpath="{.spec.ports[0].nodePort}"`
  - Alternatively, you can run the command below in a terminal (keep it open) to use localhost with the returned port number:
    - `minikube service svc-example --url`
- When using a cloud provider, you would use the cloud provider node's public IP address (`EXTERNAL-IP`) and NodePort.

In [8]:
!curl http://192.168.49.2:32410

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


## Delete the Service and Deployment

In [9]:
!kubectl delete -f manifests/nodeport.yaml
!kubectl delete -f manifests/deploy-app.yaml

service "svc-example" deleted
deployment.apps "deploy-nginx" deleted


## List Services, Deployments and Pods

- We see that the Service and Deployment (with its three Pods) have been deleted.

In [10]:
#!kubectl get svc -o wide
#!kubectl get deploy -o wide
#!kubectl get po -o wide
!kubectl get services -o wide
!kubectl get deployments -o wide
!kubectl get pods -o wide

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE   SELECTOR
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   11h   <none>
No resources found in default namespace.
No resources found in default namespace.
